In [ ]:
!pip install kafka-python s3fs boto3

In [ ]:
import os
import json
from kafka import KafkaConsumer
from json import loads, dumps
from s3fs import S3FileSystem
import boto3

In [ ]:
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_DEFAULT_REGION", "ap-south-1")

In [ ]:
s3 = S3FileSystem(
    key=AWS_ACCESS_KEY,
    secret=AWS_SECRET_KEY,
    client_kwargs={'region_name': AWS_REGION}
)

In [ ]:
consumer = KafkaConsumer(
    'demo_test',
    bootstrap_servers=['13.233.157.88:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [ ]:
for count, message in enumerate(consumer):
    try:
        s3_path = f"s3://your-bucket-name/stock_market_{count}.json"
        with s3.open(s3_path, 'w') as file:
            json.dump(message.value, file)
        print(f"Successfully wrote {s3_path}")
    except Exception as e:
        print(f"Error writing to S3: {str(e)}")

In [ ]:
for message in consumer:
    try:
        data = json.loads(message.value)
        df_received = pd.DataFrame(data)
        print(df_received)
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    break